# Exploring The Disney Datasets
## Does the popularity of songs in musicals make Disney money?
### Eric G. Dennis
### Prepared and submitted as part of the "Programming in Python For Data Science!

# Introduction

## Question of interests
For this project, I will be analysing data from Dinsey to investigate if the popularity of a featured song in a musical has any influence on the revenue that the movie generates. Many Disney movies have featured popular songs by popular artists that can be as big as the movie itself. Thinking back a few years, "Let it Go" seemed as omnipresent as the associated movie "Frozen". But does this popularity translate to cash in the bank? I hypothesize that there will be a positive correlation between song popularity and revenue.


## Dataset description

This [webpage](https://data.world/kgarrett/disney-character-success-00-16) contains five data tables containing various data about Disney. The website summarizes the data in these tables as "What are the trends in the Walt Disney Studio’s box office data? How do certain characters contribute to the success or failure of a movie?"

The Disney dataset is composed of  5  tables, disney_voice_actors.csv, disney_director.csv, disney_characters.csv, disney_revenue_year.csv, and disney_total_gross.csv . Each table is stored in a .csv file and contains different information about actors, directors, revenue, and associated characters of some Disney movies. I will be using the disney_total_gross and disney_characters tables formally described below:

* **disney_total_gross.csv**
    * This file contains information on Disney movie titles, including release dates, genre, MPAA rating, the total gross income of the film, and the gross income adjusted for inflation.
* **disney_characters**
    * This file contains information on a subset of Disney musical movies, including release date, the movie hero, the villian, and its featured song.

In order to answer our question, we need a way to quantify popularity of a song. The appearance of featured songs of movies in the US Billboard Top 100 Charts was chosen to do this with the premise that the more popular the song, the higher it will chart.
* **songs_billboard**
    * A further data table called songs_billboard has been generated from data from [here](https://www.officialcharts.com/search/singles/). Searching the featured songs provided the data on the performing artist, the peak chart position and weeks spent on charts.

# Methods and Results
The data I need to make an appropriate comparison of song popularity and revenue are the tables **total_gross**, **characters**, and **songs_billboard**.

To start, let's import the tables and look at column names and datatype info.

In [1]:
# Importing altair and pandas will be most useful here.
import altair as alt
import pandas as pd

# Import all the desired files.
total_gross = pd.read_csv("data/disney_movies_total_gross.csv", parse_dates=["release_date"])
characters = pd.read_csv("data/disney-characters.csv", parse_dates=["release_date"])
songs_billboard = pd.read_excel("data/Eric data/songs_billboard.xlsx", sheet_name="Sheet1")

The 'release_date' in both **total_gross** and **characters** is likely to be important in our processing, so that column has been read in from the resepective CSV as a datetime64 dtype.

In [2]:
total_gross.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,1940-02-09,Adventure,G,"$84,300,000","$2,188,229,052"
2,Fantasia,1940-11-13,Musical,G,"$83,320,000","$2,187,090,808"
3,Song of the South,1946-11-12,Adventure,G,"$65,000,000","$1,078,510,579"
4,Cinderella,1950-02-15,Drama,G,"$85,000,000","$920,608,730"


In [3]:
total_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   movie_title               579 non-null    object        
 1   release_date              579 non-null    datetime64[ns]
 2   genre                     562 non-null    object        
 3   MPAA_rating               523 non-null    object        
 4   total_gross               579 non-null    object        
 5   inflation_adjusted_gross  579 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 27.3+ KB


In [4]:
characters.head()

,movie_title,release_date,hero,villian,song
0,\nSnow White and the Seven Dwarfs,1937-12-21,Snow White,Evil Queen,Some Day My Prince Will Come
1,\nPinocchio,1940-02-07,Pinocchio,Stromboli,When You Wish upon a Star
2,\nFantasia,1940-11-13,NaN,Chernabog,NaN
3,Dumbo,1941-10-23,Dumbo,Ringmaster,Baby Mine
4,\nBambi,1942-08-13,Bambi,Hunter,Love Is a Song


In [5]:
characters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   movie_title   56 non-null     object        
 1   release_date  56 non-null     datetime64[ns]
 2   hero          52 non-null     object        
 3   villian       46 non-null     object        
 4   song          47 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.3+ KB


In [6]:
songs_billboard.head()

,Movie,Artist,Song,Peak,WoC
0,Big Hero 6,Fallout Boy,Immortals,84,3
1,Moana,Alessia Cara,How Far Ill Go,49,13
2,Frozen,Idina Menzel,Let It Go,11,70
3,Brother Bear,Phil Collins,Look Through My Eyes,61,2
4,Tarzan,Phil Collins,You'll Be In My Heart,17,8


In [7]:
songs_billboard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   9 non-null      object
 1   Artist  9 non-null      object
 2   Song    9 non-null      object
 3   Peak    9 non-null      int64 
 4   WoC     9 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 488.0+ bytes


I will first work with the **characters** dataframe. I am interested in movies that contain songs. I will clean and filter the dataframe in order to produce a dataframe with Movie Titles only containing Songs.

There are three steps to this cleaning: Stripping extra characters; dropping columns; dropping null values. 

In [8]:
# Step 1: Stripping extra characters.
characters_stripped = characters.assign(movie_title=characters["movie_title"].str.strip("\n"))
characters_stripped = characters_stripped.assign(song=characters_stripped["song"].str.strip("\n"))
characters_stripped.head()

,movie_title,release_date,hero,villian,song
0,Snow White and the Seven Dwarfs,1937-12-21,Snow White,Evil Queen,Some Day My Prince Will Come
1,Pinocchio,1940-02-07,Pinocchio,Stromboli,When You Wish upon a Star
2,Fantasia,1940-11-13,NaN,Chernabog,NaN
3,Dumbo,1941-10-23,Dumbo,Ringmaster,Baby Mine
4,Bambi,1942-08-13,Bambi,Hunter,Love Is a Song


In [9]:
characters_stripped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   movie_title   56 non-null     object        
 1   release_date  56 non-null     datetime64[ns]
 2   hero          52 non-null     object        
 3   villian       46 non-null     object        
 4   song          47 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.3+ KB


In [10]:
# Steps 2 and 3.Lets make a new dataframe that drops columns 'hero' and 'villian' then drop any rows withouts songs
movies_songs = characters_stripped.drop(columns=["hero", "villian"]).dropna(subset=["song"])
movies_songs.reset_index()

# There is one erroneous piece of data, 'The Rescuers Down Under" shows an empty cell, not NaN for a song.
# It should be dropped.

movies_songs_cleaned = movies_songs.drop(28, axis=0)

# This should give a dataframe containing 46 movie entries and three columns.
# Checking the shape confirms this.
movies_songs_cleaned.shape

(46, 3)

In [11]:
# A quick check on movie_songs_cleaned info confirms we have no null values in our song column.
movies_songs_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 55
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   movie_title   46 non-null     object        
 1   release_date  46 non-null     datetime64[ns]
 2   song          46 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 1.4+ KB


In [12]:
movies_songs_cleaned.head()

,movie_title,release_date,song
0,Snow White and the Seven Dwarfs,1937-12-21,Some Day My Prince Will Come
1,Pinocchio,1940-02-07,When You Wish upon a Star
3,Dumbo,1941-10-23,Baby Mine
4,Bambi,1942-08-13,Love Is a Song
5,Saludos Amigos,1943-02-06,Saludos Amigos


The characters dataframe is now processed to only include movies with songs associated. A merge with **total_gross** is desired to compare songs with with movie revenue.

The **total_gross** dataframe needs some wrangling. Before merging, I will drop the columns 'release date', 'genre', and 'MPAA_rating' as they are not useful in answering our question. 

We also want the 'total_gross' and 'inflation_adjusted_gross' columns as integer dtypes so that we can plot the values in the future.

In [13]:
total_gross_dropped = total_gross.drop(columns=["release_date", "genre", "MPAA_rating"])
total_gross_dropped.reset_index()

total_gross_fixed = total_gross_dropped.assign(total_gross=total_gross_dropped["total_gross"].str.strip("$").str.replace(",", ""))
total_gross_fixed = total_gross_fixed.assign(inflation_adjusted_gross=total_gross_fixed["inflation_adjusted_gross"].str.strip("$").str.replace(",", ""))
total_gross_fixed = total_gross_fixed.assign(total_gross=total_gross_fixed["total_gross"].astype("int"))
total_gross_fixed = total_gross_fixed.assign(inflation_adjusted_gross=total_gross_fixed["inflation_adjusted_gross"].astype("int"))

total_gross_fixed.info()

# That was tough, but we have the money-related columns as integer dtypes now.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   movie_title               579 non-null    object
 1   total_gross               579 non-null    int64 
 2   inflation_adjusted_gross  579 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 13.7+ KB


To produce the final dataframe of interest, two merges will be needed. The first to merge **movies_songs_cleaned** and **total_gross_fixed**. The second will be to merge the **songs_billboard** with the dataframe from the first merge. It makes sense by the DRY principle to make a custom script for such a merge based on the shared column of 'movie_titles'.

In [14]:
# Import the custom_merge_script.
import custom_merge_script as ps

# Let's use the custom script to merge movie_songs_cleaned and total_gross_fixed.
songs_merged = ps.custom_merge(movies_songs_cleaned, total_gross_fixed, 'left', 'movie_title')
songs_merged
# Success!

,movie_title,release_date,song,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Some Day My Prince Will Come,184925485.0,5.228953e+09
1,Pinocchio,1940-02-07,When You Wish upon a Star,84300000.0,2.188229e+09
2,Dumbo,1941-10-23,Baby Mine,NaN,NaN
3,Bambi,1942-08-13,Love Is a Song,NaN,NaN
4,Saludos Amigos,1943-02-06,Saludos Amigos,NaN,NaN
5,Melody Time,1948-05-27,Little Toot,NaN,NaN
6,The Adventures of Ichabod and Mr. Toad,1949-10-05,The Merrily Song,NaN,NaN
7,Cinderella,1950-02-15,Bibbidi-Bobbidi-Boo,85000000.0,9.206087e+08
8,Cinderella,1950-02-15,Bibbidi-Bobbidi-Boo,201151353.0,2.011514e+08
9,Alice in Wonderland,1951-07-28,The Unbirthday Song,334191110.0,3.570635e+08


Some processing of the merged dataframe is needed, but before that it's a good time to mention some details from **songs_billboard**. 
While we are looking at this dataframe, lets rename 'Movie" to 'movie_title' to aid in consitency for merging.

In [15]:
songs_billboard = songs_billboard.rename(columns={"Movie": "movie_title"})
songs_billboard

,movie_title,Artist,Song,Peak,WoC
0,Big Hero 6,Fallout Boy,Immortals,84,3
1,Moana,Alessia Cara,How Far Ill Go,49,13
2,Frozen,Idina Menzel,Let It Go,11,70
3,Brother Bear,Phil Collins,Look Through My Eyes,61,2
4,Tarzan,Phil Collins,You'll Be In My Heart,17,8
5,Hercules,Michael Bolton,Go The Distance,14,6
6,The Lion King,Elton John,Circle of Life,11,12
7,Aladdin,Peabo Bryson and Regina Belle,A Whole New World,12,12
8,The Little Mermaid,Sebastian C,Under the Sea,90,2


This dataframe does show a severe limitation to the conclusions we can make on our question. Only 9 songs appeared on the US Billboard Charts(1), so the data to plot to find any correlation between song popularity and the films earning will be limited. However, it is sufficient for the purpose of this project to move forward.

Comparing **songs_merged** with **songs_billboard**, no movie made prior to 1989 charted on the US Billboard, and hence is not present in the **songs_billboard** dataframe. As such, **songs_merged** can be filtered to remove movies made prior to 1989 and will reduce the size of the dataframe prior to merging with **songs_billboard**.

In [16]:
# Looking at the dtypes of songs_merged reveals 'release_date' is still a datetime64 dtype.
# This can be converted to just show the year.
song_merged_years = songs_merged.assign(year=songs_merged['release_date'].dt.year).drop(columns='release_date')
song_merged_years.dtypes

# The new dataframe contains the year column as an int64 dtype.

# Now we can remove the years before 1989 by filtering for greater than 1988.

song_filtered_years = song_merged_years[song_merged_years["year"] > 1988]
song_filtered_years

,movie_title,song,total_gross,inflation_adjusted_gross,year
25,The Little Mermaid,Under the Sea,111543479.0,223726012.0,1989
26,Beauty and the Beast,Be Our Guest,218951625.0,363017667.0,1991
27,Aladdin,A Whole New World,217350219.0,441969178.0,1992
28,The Lion King,Circle of Life,422780140.0,761640898.0,1994
29,Pocahontas,Colors of the Wind,141579773.0,274370957.0,1995
30,The Hunchback of Notre Dame,God Help the Outcasts,100138851.0,190988799.0,1996
31,Hercules,Go the Distance,99112101.0,182029412.0,1997
32,Mulan,I'll Make a Man Out of You,120620254.0,216807832.0,1998
33,Tarzan,You'll Be in My Heart,171091819.0,283900254.0,1999
34,The Emperor's New Groove,My Funny Friend and Me,89296573.0,136789252.0,2000


Now for the fun part. A second use of the custom merge can be used here to combine the **songs_billboard** and **song_filtered_years** dataframes so we can generate some plots to compare revenues vs popularity. 

The 'inflation_ajusted_gross' makes the most sense to use in comparison plots as it provides a reasonable normalization for data between different years for the release of the respective movies.

In [17]:
billboard_gross_merged = ps.custom_merge(songs_billboard,song_filtered_years, 'left', 'movie_title')

# Success, now lets sort it by inflation.
billboard_gross_sorted = billboard_gross_merged.sort_values(by='inflation_adjusted_gross', ascending=False).drop(columns=['song'])
billboard_gross_sorted.reset_index()

,index,movie_title,Artist,Song,Peak,WoC,total_gross,inflation_adjusted_gross,year
0,6,The Lion King,Elton John,Circle of Life,11,12,422780140.0,761640898.0,1994
1,7,Aladdin,Peabo Bryson and Regina Belle,A Whole New World,12,12,217350219.0,441969178.0,1992
2,2,Frozen,Idina Menzel,Let It Go,11,70,400738009.0,414997174.0,2013
3,4,Tarzan,Phil Collins,You'll Be In My Heart,17,8,171091819.0,283900254.0,1999
4,1,Moana,Alessia Cara,How Far Ill Go,49,13,246082029.0,246082029.0,2016
5,0,Big Hero 6,Fallout Boy,Immortals,84,3,222527828.0,229249222.0,2014
6,8,The Little Mermaid,Sebastian C,Under the Sea,90,2,111543479.0,223726012.0,1989
7,5,Hercules,Michael Bolton,Go The Distance,14,6,99112101.0,182029412.0,1997
8,3,Brother Bear,Phil Collins,Look Through My Eyes,61,2,85336277.0,119218333.0,2003


Time to submit this assignment is running out. Making histograms by movie for inflation_ajusted_gross Peak is a quick and dirty method for vizualising the data for a first look for any kind of trend.

In [18]:
gross_plot = (
    alt.Chart(billboard_gross_sorted, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("movie_title:N", sort="-y", title="Movie Name"),
        y=alt.Y(
            "inflation_adjusted_gross:Q", title="Gross Revenue Adjusted for Inflation"
        ),
    )
    .properties(title="Gross revenue for each movie title")
)

gross_plot

alt.Chart(...)

In [19]:
peak_plot = (
    alt.Chart(billboard_gross_sorted, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("movie_title:N", sort="y", title="Movie Name"),
        y=alt.Y("Peak:Q", title="Peak position on Charts"),
    )
    .properties(title="Peak on Billboard Chart for each movie title")
)

peak_plot
# Note: This chart has been sorted by descending order as the lower peak represents a more popular song.

alt.Chart(...)

In [20]:
woc_plot = (
    alt.Chart(billboard_gross_sorted, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("movie_title:N", sort="-y", title="Movie Name"),
        y=alt.Y("WoC:Q", title="Weeks on Charts"),
    )
    .properties(title="Weeks in Billboard Chart for each movie title")
)

woc_plot

alt.Chart(...)

There is something here in these data worth discussing.

# Discussions

In this analysis, I have made an initial comparison of the popularity of a featured song in a Disney musical to how much money the film generated. Comparing the histograms generated, there is a trend that the movies with the most popular songs (The Lion King, Frozen, Aladdin) according to chart peak (around 10) and weeks on chart were the highest grossing movies. Conversly, for the lowest grossing films from the dataset (Brother Bear, The Little Mermaid), the feature songs peaked no better than 60 and spent fewer weeks in the charts. To address the inital question, this analysis suggests that there is a potential positive correlation between movie revenue and the popularity of a featured song. To be honest, I did not expect any kind of trend to emerge.

This conclusion is very limited given the dataset size and the visual nature of the histogram comparison. A better comparison of the data could have been a scatter plot with multiple y-axis variables (eg gross and peak vs movie title). However, I am running out of time to submit this report and multiple axes plots look complicated. Looks like this might be a good candidate project to take forward into the "Exploratory Data Visualization" course.

Quantifying popularity by chart success is only one way to to explore the question I posed in this project. Comparing song revenue to movie revenue or the number of times a song is aired (such as radio or youtube) to movie revenue are also viable methods to explore the question.

# References

I cannot take all the credit for the preceding work. I dove into this project head first before I checked the depth of the water and have had a lot of help along the way. As such, I would like to acknowledge the following resources.

## Resources used
* [Disney Data Source](https://data.world/kgarrett/disney-character-success-00-16)
    * The Disney dataframes used in this work was curated by **Kelly Garrett**.
* [Billboard Data Source](https://www.officialcharts.com/search/singles/)
    * The Billboard dataframe used in this work was produced by myself with help of searching each song by title using this webpage.
* [helper data source](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)
    * The helper data for the custom merge function was taken from this website.
* **Question Of Interest**
    * The question of interest was inspired through discussions with **Jessica Wood**, a former student in the course.
    * A big thank you to **Socorro Dominguez** for encouraging me to tackle this this question and for our helpful email discussions.
* **Acknowledgements** 
    * Thanks to the sample report (and its authors). It was invaluable for the layout and expectations of the report. I also learned a lot of formatting in markdown from it.
    * Thanks to everyone on the course Piazza community for your insights and inputs into each others projects and assignments. Its been an enjoyable and inclusive place to learn.
    * Thanks to the Python team at UBC EL for putting together an overall very enjoyable and well organized course. I revisted the module slideshows on many occasions while writing this project.
    * The many, many, many forums and document webpages I visited throughout the course to help me through my coding errors and doubts.

### Notes
(1): In the case of some songs, multiple different version releases could be found in the Billboard Chart search. In these cases, the data in the dataframe reflects the version of the song that was included in the film soundtrack.